
## Libraries import

In [1]:
from pysent import AspectAnotator
import pysent.aspect_annotators.extrassifiers as extrassifiers
from pysent.data_structures import concat_results, transform_output

import pandas as pd

C:\Users\welki\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html


No CUDA GPU found in your device
[2023-12-10 09:46:46] (2.3.4) PyABSA(2.3.4): If your code crashes on Colab, please use the GPU runtime. Then run "pip install pyabsa[dev] -U" and restart the kernel.
Or if it does not work, you can use v1.x versions, e.g., pip install pyabsa<2.0 -U




Try to downgrade transformers<=4.29.0.






C:\Users\welki\AppData\Local\Programs\Python\Python311\Lib\multiprocessing\pool.py:268: ResourceWarning: unclosed running multiprocessing pool <multiprocessing.pool.Pool state=RUN pool_size=1>


## Load and filter the PolEmo dataset

In [2]:
polemo = pd.read_csv("../data/polemo.csv", index_col=0)

In [3]:
polemo = polemo.loc[polemo['review'].str.len() < 375,["review"]].reset_index(drop=True)

## Annotate the filtered PolEmo dataset using pyabsa "extraclassifier"

In [4]:
polemo_reviews_list = polemo.review.to_list()

In [5]:
pyabsa_full = extrassifiers.PyabsaExtrassifier()

[2023-12-10 09:46:57] (2.3.4) Please specify the task code, e.g. from pyabsa import TaskCodeOption
[2023-12-10 09:46:58] (2.3.4) Downloading checkpoint:multilingual 
[2023-12-10 09:46:58] (2.3.4) Notice: The pretrained model are used for testing, it is recommended to train the model on your own custom datasets
[2023-12-10 09:46:58] (2.3.4) Checkpoint already downloaded, skip
[2023-12-10 09:46:58] (2.3.4) Load aspect extractor from checkpoints\ATEPC_MULTILINGUAL_CHECKPOINT
[2023-12-10 09:46:58] (2.3.4) config: checkpoints\ATEPC_MULTILINGUAL_CHECKPOINT\fast_lcf_atepc.config
[2023-12-10 09:46:58] (2.3.4) state_dict: checkpoints\ATEPC_MULTILINGUAL_CHECKPOINT\fast_lcf_atepc.state_dict
[2023-12-10 09:46:58] (2.3.4) model: None
[2023-12-10 09:46:58] (2.3.4) tokenizer: checkpoints\ATEPC_MULTILINGUAL_CHECKPOINT\fast_lcf_atepc.tokenizer
[2023-12-10 09:46:58] (2.3.4) Set Model Device: cpu
[2023-12-10 09:46:58] (2.3.4) Device Name: Unknown


C:\Users\welki\AppData\Local\Programs\Python\Python311\Lib\site-packages\transformers\convert_slow_tokenizer.py:473: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.


In [6]:
annotator = AspectAnotator([pyabsa_full])

In [7]:
annotations = annotator.annotate(polemo_reviews_list)

preparing apc inference dataloader: 100%|█████████████████████████████████████████| 2489/2489 [00:14<00:00, 167.32it/s]
C:\Users\welki\AppData\Roaming\Python\Python311\site-packages\pyabsa\tasks\AspectTermExtraction\prediction\aspect_extractor.py:566: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ..\torch\csrc\utils\tensor_new.cpp:264.)
classifying aspect sentiments: 100%|███████████████████████████████████████████████████| 78/78 [12:39<00:00,  9.74s/it]


## Restructure output

In [8]:
df_out = transform_output(reviews=polemo_reviews_list,
                          annotations=annotations)

In [9]:
df_out.to_csv("../data/polemo_labelled.csv", index=False)

## Manual tests

### Analayze 10 sample aspects

In [10]:
for index, row in df_out.sample(10).iterrows():
    print("#################")
    print(row['review'])
    print("######")
    print(f"{row['aspect']} -> {row['label']}")

#################
Hotel bardzo dobrze położony . Pokój przestronny , ładnie urządzony . Posiłki dobre , ale bez rewelacji . WiFi właściwie nie działa . Duży aquapark to na pewno atrakcja . Dopłata za korzystanie z lodówki w pokoju , brak chusteczek higienicznych w łazience . Zepsutą suszarkę w łazience wymieniono dopiero następnego dnia . Ogólnie ok , ale szału nie ma .
######
WiFi -> negative
#################
Hotel podaje fałszywy adres na portalach rezerwacyjnych naprawdę znajduje się w miejscowości i odległej o 7 km od Gdańska drogowskaz stoi obok hotelu , z koszmarnym dojazdem ( 1 , 5 h jak masz szczęście ) mimo , ze rozmawiał em o tym z obsługą w hotelu nie widzą oni nic złego w oszukiwaniu klientów , Po za tym niewygodne łóżka , drogie posiłki
######
obsługą -> negative
#################
Mam sentyment do tego miejsca spędził em w tym hotelu wiele nocy . Hotel jest świetnie ulokowany - w centrum miasta . Jak na tą lokalizację ceny są dobre . Śniadania do przyjęcia ale to już nie 

### Analyze 5 sample whole reviews

In [11]:
sample_whole_reviews = df_out.loc[df_out.review.isin(df_out.sample(5).review.to_list()),:]

In [12]:
for index, row in sample_whole_reviews.iterrows():
    print("#################")
    print(row['review'])
    print("######")
    print(f"{row['aspect']} -> {row['label']}")

#################
Spędzone tu 4 dni będę wspominała miło . Profesjonalny , miły , uśmiechnięty i przede wszystkim pomocny personel . Pokoje czyste zadbane komfortowe . Posiłki smaczne świeże po prostu mmm pycha : ) Bardzo blisko ( 2 - 3 min pieszo ) Galeria Łódzka , żabka . nie ma problemu z komunikacją : ) wszędzie się dotrze . Polecam bardzo fajne miejsce
######
personel -> positive
#################
Spędzone tu 4 dni będę wspominała miło . Profesjonalny , miły , uśmiechnięty i przede wszystkim pomocny personel . Pokoje czyste zadbane komfortowe . Posiłki smaczne świeże po prostu mmm pycha : ) Bardzo blisko ( 2 - 3 min pieszo ) Galeria Łódzka , żabka . nie ma problemu z komunikacją : ) wszędzie się dotrze . Polecam bardzo fajne miejsce
######
Pokoje -> positive
#################
Spędzone tu 4 dni będę wspominała miło . Profesjonalny , miły , uśmiechnięty i przede wszystkim pomocny personel . Pokoje czyste zadbane komfortowe . Posiłki smaczne świeże po prostu mmm pycha : ) Bardzo blis

### Basic characteristics of the prepared dataset

In [13]:
print(f"The prepared dataset consists of {polemo.shape[0]} reviews")
print(f"The prepared dataset consists of {df_out.shape[0]} aspects")

The prepared dataset consists of 992 reviews
The prepared dataset consists of 2489 aspects
